<a href="https://colab.research.google.com/github/GuiMM27/Sprint4-DYN/blob/main/Sprint4_DYN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SPRINT 4 - Programação Dinâmica para Otimização de Estoque de Insumos

1. Setup e Definição dos Parâmetros do Modelo

In [1]:
import numpy as np
import sys
# Aumenta o limite de recursão para evitar Runtime Error em testes maiores
sys.setrecursionlimit(2000)

# =================================================================
# PARÂMETROS DO MODELO
# =================================================================

T = 7            # Horizonte de tempo (Dias de planejamento)
M = 20           # Capacidade Máxima do Estoque (Limite superior para os estados)
ESTOQUE_INICIAL_GLOBAL = 5 # Estoque no início do Dia 1 (t=0)

# Custos
K = 50           # Custo Fixo de Pedido (Setup cost)
c = 10           # Custo Unitário de Compra
h = 2            # Custo Unitário de Armazenamento por dia (Holding cost)
p = 100          # Custo Unitário de Penalidade por Falta (Shortage cost)

# Consumo Diário Simulado (Demanda C_t para t=0 a T-1)
# Este é o dado que o modelo de PD usa para otimizar as decisões
CONSUMO = [5, 8, 3, 10, 4, 7, 6]

# =================================================================

2. Função de Transição e Custo (Função de Recorrência)

In [2]:
def calcular_custo_e_transicao(estoque_inicial, quantidade_pedida, consumo_do_dia):
    """
    Calcula o custo imediato incorrido no dia t e o estado do estoque para o dia t+1.

    Args:
        estoque_inicial (int): Estoque no início do dia S_t.
        quantidade_pedida (int): Decisão D_t.
        consumo_do_dia (int): Consumo C_t.

    Returns:
        tuple: (custo_total_diario, estoque_final_s_t_mais_1)
    """

    # 1. Custo de Pedido
    custo_pedido = c * quantidade_pedida
    if quantidade_pedida > 0:
        custo_pedido += K

    estoque_disponivel = estoque_inicial + quantidade_pedida

    # 2. Custo de Falta (Penalidade)
    falta = max(0, consumo_do_dia - estoque_disponivel)
    custo_falta = p * falta

    # 3. Novo Estoque (Estado de Transição S_{t+1})
    estoque_final = max(0, estoque_disponivel - consumo_do_dia)

    # 4. Custo de Armazenamento/Estoque
    custo_estoque = h * estoque_final

    custo_total = custo_pedido + custo_falta + custo_estoque

    return custo_total, estoque_final

3. Versão 1: Recursiva com Memorização (Top-Down)

In [3]:
# Tabela de memorização (Memoization Table) para o custo
MEMO_CUSTO = {}
# Tabela de memorização para a política ótima (decisão)
MEMO_POLITICA = {}

def pd_recursiva_top_down(t, estoque_inicial):
    """
    Implementa a Programação Dinâmica de forma recursiva (Top-Down).
    Calcula o custo mínimo do dia t até o final do horizonte T.

    Args:
        t (int): Índice do dia (0 a T-1).
        estoque_inicial (int): Estoque S_t no início do dia t.

    Returns:
        float: Custo mínimo acumulado a partir de t.
    """

    # 1. Caso Base: Fim do horizonte
    if t == T:
        return 0.0 # Custo zero após o último dia

    # 2. Caso de Memorização (Lookup)
    if (t, estoque_inicial) in MEMO_CUSTO:
        return MEMO_CUSTO[(t, estoque_inicial)]

    consumo_do_dia = CONSUMO[t]
    min_custo_acumulado = float('inf')
    melhor_decisao = -1

    # 3. Iterar sobre as Decisões (D_t)
    # A quantidade pedida vai de 0 até o limite M-estoque_inicial
    for D_t in range(M - estoque_inicial + 1):

        # 3.1. Calcular o Custo Imediato e o Próximo Estado (S_{t+1})
        custo_imediato, estoque_final = calcular_custo_e_transicao(
            estoque_inicial, D_t, consumo_do_dia
        )

        # 3.2. Chamar Recursivamente o Custo Futuro
        custo_recursivo = pd_recursiva_top_down(t + 1, estoque_final)

        # 3.3. Custo Total para esta decisão
        custo_total_atual = custo_imediato + custo_recursivo

        # 4. Atualizar o Custo Mínimo e a Política
        if custo_total_atual < min_custo_acumulado:
            min_custo_acumulado = custo_total_atual
            melhor_decisao = D_t

    # 5. Salvar na Tabela de Memorização
    MEMO_CUSTO[(t, estoque_inicial)] = min_custo_acumulado
    MEMO_POLITICA[(t, estoque_inicial)] = melhor_decisao

    return min_custo_acumulado

# Inicializa as tabelas para a execução
MEMO_CUSTO.clear()
MEMO_POLITICA.clear()

custo_recursivo_final = pd_recursiva_top_down(0, ESTOQUE_INICIAL_GLOBAL)

print(f"--- 1. Versão Recursiva com Memorização (Top-Down) ---")
print(f"Custo Mínimo Total: R$ {custo_recursivo_final:.2f}")

--- 1. Versão Recursiva com Memorização (Top-Down) ---
Custo Mínimo Total: R$ 556.00


4. Versão 2: Iterativa (Bottom-Up)

In [4]:
def pd_iterativa_bottom_up(estoque_inicial_global):
    """
    Implementa a Programação Dinâmica de forma iterativa (Bottom-Up).
    Preenche uma tabela de custos (DP_Table) do final (t=T) para o início (t=0).

    Args:
        estoque_inicial_global (int): Estoque S_0 no início do Dia 1.

    Returns:
        tuple: (custo_minimo_total, DP_Table, Politica_Table)
    """
    # DP_Table[t][s]: Custo mínimo para o dia t (0 a T) com estoque inicial s
    DP_Table = np.full((T + 1, M + 1), float('inf'))

    # Politica_Table[t][s]: Melhor decisão D_t para o dia t com estoque s
    Politica_Table = np.full((T, M + 1), -1, dtype=int)

    # 1. Caso Base: Dia t = T (após o último dia)
    DP_Table[T, :] = 0.0

    # 2. Iterar de trás para frente (t = T-1 até 0)
    for t in range(T - 1, -1, -1):
        consumo_do_dia = CONSUMO[t]

        # 3. Iterar sobre todos os Estados (Estoque Inicial S_t)
        for estoque_inicial in range(M + 1):

            min_custo_acumulado = float('inf')
            melhor_decisao = -1

            # 4. Iterar sobre todas as Decisões (Quantidade Pedida D_t)
            for D_t in range(M - estoque_inicial + 1):

                # 4.1. Calcular Custo Imediato e Próximo Estado
                custo_imediato, estoque_final = calcular_custo_e_transicao(
                    estoque_inicial, D_t, consumo_do_dia
                )

                # 4.2. Buscar Custo Futuro na DP_Table (já calculado na iteração t+1)
                custo_futuro = DP_Table[t + 1, estoque_final]

                custo_total_atual = custo_imediato + custo_futuro

                # 5. Atualizar o Custo Mínimo
                if custo_total_atual < min_custo_acumulado:
                    min_custo_acumulado = custo_total_atual
                    melhor_decisao = D_t

            # 6. Salvar na DP_Table e Politica_Table
            DP_Table[t, estoque_inicial] = min_custo_acumulado
            Politica_Table[t, estoque_inicial] = melhor_decisao

    # O resultado é o custo mínimo no dia 0 com o estoque inicial dado
    custo_iterativo_final = DP_Table[0, estoque_inicial_global]

    return custo_iterativo_final, DP_Table, Politica_Table


# Execução da Versão Iterativa
custo_iterativo_final, DP_Table, Politica_Table = pd_iterativa_bottom_up(ESTOQUE_INICIAL_GLOBAL)

print(f"\n--- 2. Versão Iterativa (Bottom-Up) ---")
print(f"Custo Mínimo Total: R$ {custo_iterativo_final:.2f}")


--- 2. Versão Iterativa (Bottom-Up) ---
Custo Mínimo Total: R$ 556.00


5. Garantia de Resultados e Análise da Política Ótima

In [5]:
# =================================================================
# GARANTIA DE RESULTADOS E ANÁLISE DA POLÍTICA ÓTIMA
# =================================================================

# 1. Garantir que ambas produziram os mesmos resultados
print("\n--- 3. Verificação de Equivalência ---")
if abs(custo_recursivo_final - custo_iterativo_final) < 1e-6:
    print(f"✅ Sucesso: Ambas as versões (R$ {custo_recursivo_final:.2f}) são equivalentes.")
else:
    print(f"❌ Aviso: Resultados divergentes (Rec: {custo_recursivo_final:.2f} | Iter: {custo_iterativo_final:.2f})")

# 2. Demonstração da Política Ótima (Para validação do modelo)
print("\n--- 4. Política Ótima de Reposição (Caminho Solução) ---")
print("Dia | Estoque Inicial | Consumo | Decisão (Pedido) | Estoque Final")

estoque_atual = ESTOQUE_INICIAL_GLOBAL
custo_total_politica = 0

for t in range(T):
    # A decisão é lida da tabela de política gerada pela versão iterativa
    decisao_otima = Politica_Table[t, estoque_atual]
    consumo_do_dia = CONSUMO[t]

    custo_imediato, estoque_final = calcular_custo_e_transicao(
        estoque_atual, decisao_otima, consumo_do_dia
    )

    custo_total_politica += custo_imediato

    print(f"{t+1:<3} | {estoque_atual:<15} | {consumo_do_dia:<7} | {decisao_otima:<16} | {estoque_final}")

    estoque_atual = estoque_final

print(f"\nCusto Total Reconstruído (deve ser igual ao mínimo): R$ {custo_total_politica:.2f}")


--- 3. Verificação de Equivalência ---
✅ Sucesso: Ambas as versões (R$ 556.00) são equivalentes.

--- 4. Política Ótima de Reposição (Caminho Solução) ---
Dia | Estoque Inicial | Consumo | Decisão (Pedido) | Estoque Final
1   | 5               | 5       | 0                | 0
2   | 0               | 8       | 11               | 3
3   | 3               | 3       | 0                | 0
4   | 0               | 10      | 14               | 4
5   | 4               | 4       | 0                | 0
6   | 0               | 7       | 13               | 6
7   | 6               | 6       | 0                | 0

Custo Total Reconstruído (deve ser igual ao mínimo): R$ 556.00


6. Relatório e Explicação do Algoritmo (Documentação)